<a href="https://colab.research.google.com/github/mbagherian/CMMC-CTMC/blob/master/Attention1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer, AdamW

In [ ]:
!pip install tqdm
import torch
from tqdm.notebook import tqdm

In [ ]:
# Loading dataf dataset
#dataf_path = 'C:/Users/Ankita/Desktop/RA/Attention/Transformer/DATAF.csv'
dataf_path = '/content/DATAF.csv'
dataf = pd.read_csv(dataf_path)



In [ ]:
# Loading Forex_Comp dataset
forex_comp_path = '/content/Forex_Comp.csv'
forex_comp = pd.read_csv(forex_comp_path)


In [ ]:
print("First few rows of DATAF.csv:")
display(dataf.head())

print("First few rows of Forex_Comp.csv:")
display(forex_comp.head())

First few rows of DATAF.csv:


,Date,DFFR,EURINF,USAINF,EURLTINT,USALTINT,EURUNEMP,USAUNEMP
0,1/3/2000,5.43,1.9,2.7,5.7,6.66,9.4,4.0
1,1/4/2000,5.38,1.9,2.7,5.7,6.66,9.4,4.0
2,1/5/2000,5.41,1.9,2.7,5.7,6.66,9.4,4.0
3,1/6/2000,5.54,1.9,2.7,5.7,6.66,9.4,4.0
4,1/7/2000,5.61,1.9,2.7,5.7,6.66,9.4,4.0


First few rows of Forex_Comp.csv:


,Subset,Model,MAE,RMSE
0,Train,Dual-input LSTM,0.007749,0.010368
1,Train,Dual-input LSTM,0.007640,0.010263
2,Train,Dual-input LSTM,0.008312,0.010969
3,Train,Dual-input LSTM,0.007878,0.010565
4,Train,Dual-input LSTM,0.007898,0.010557


In [ ]:
# Split DATAF.csv into training and testing sets
dataf_train, dataf_test = train_test_split(dataf, test_size=0.2, random_state=42)


In [ ]:
# Split Forex_Comp.csv into training and testing sets
forex_comp_train, forex_comp_test = train_test_split(forex_comp, test_size=0.2, random_state=42)


In [ ]:
# Display the first few rows of the training and testing sets
print("First few rows of DATAF training set:")
display(dataf_train.head())

print("First few rows of DATAF testing set:")
display(dataf_test.head())

First few rows of DATAF training set:


,Date,DFFR,EURINF,USAINF,EURLTINT,USALTINT,EURUNEMP,USAUNEMP
5652,9/1/2021,0.08,3.4,5.4,0.36,1.30,7.6,4.7
1310,1/10/2005,2.26,1.9,3.0,3.92,4.22,9.2,5.3
1109,4/2/2004,1.00,2.1,2.3,4.51,4.35,9.3,5.6
100,5/22/2000,6.52,1.8,3.2,5.52,6.44,9.0,3.8
4204,2/12/2016,0.38,-0.1,1.0,1.23,1.75,10.4,4.9


First few rows of DATAF testing set:


,Date,DFFR,EURINF,USAINF,EURLTINT,USALTINT,EURUNEMP,USAUNEMP
5582,5/26/2021,0.06,2.0,5.0,0.48,1.58,8.1,5.8
2337,12/17/2008,0.12,1.6,0.1,3.83,2.13,7.7,7.3
1483,9/8/2005,3.49,2.6,4.7,3.40,4.20,9.1,5.0
561,2/26/2002,1.77,2.5,1.1,5.24,4.91,8.4,5.7
2195,6/2/2008,2.06,3.9,5.0,4.96,3.99,7.1,5.6


In [ ]:
print("First few rows of Forex_Comp training set:")
display(forex_comp_train.head())

print("First few rows of Forex_Comp testing set:")
display(forex_comp_test.head())

First few rows of Forex_Comp training set:


,Subset,Model,MAE,RMSE
18,Train,Dual-input LSTM,0.012811,0.011173
203,Train,Fund+Tech LSTM,0.026559,0.019721
351,Test,Fundamental LSTM,0.075204,0.088885
275,Test,Fund+Tech LSTM,0.016004,0.022318
63,Test,Dual-input LSTM,0.005952,0.007678


First few rows of Forex_Comp testing set:


,Subset,Model,MAE,RMSE
285,Test,Fund+Tech LSTM,0.017875,0.025448
281,Test,Fund+Tech LSTM,0.016643,0.022752
33,Train,Dual-input LSTM,0.022911,0.016416
211,Train,Fund+Tech LSTM,0.033117,0.042716
93,Test,Dual-input LSTM,0.009486,0.012297


In [ ]:

# Assuming `dataf` is already defined and loaded
# Splitting the dataset
train_df, test_df = train_test_split(dataf, test_size=0.2, random_state=42)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.max_len = 128

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        inputs = self.tokenizer(
            text=str(row.drop('DFFR').values.tolist()),
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target = torch.tensor(row['DFFR'], dtype=torch.float)
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'target': target
        }

# Creating DataLoaders
batch_size = 16
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
class TransformerRegressor(torch.nn.Module):
    def __init__(self):
        super(TransformerRegressor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        return linear_output

# Initialize the model, optimizer, and loss function
model = TransformerRegressor()
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()

# Training function with progress bar
def train_model(model, train_loader, optimizer, criterion, device, epoch):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Training Epoch {epoch+1}')
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['target'].to(device).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({'batch_loss': loss.item()})
    return total_loss / len(train_loader)

# Evaluation function with progress bar
def evaluate_model(model, test_loader, criterion, device, epoch):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(test_loader, desc=f'Evaluating Epoch {epoch+1}')
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device).unsqueeze(1)
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            progress_bar.set_postfix({'batch_loss': loss.item()})
    return total_loss / len(test_loader)

# Training loop with nested progress bars
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 3
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_loss = train_model(model, train_loader, optimizer, criterion, device, epoch)
    val_loss = evaluate_model(model, test_loader, criterion, device, epoch)
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

# Save the model
torch.save(model.state_dict(), 'transformer_regressor.pth')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Evaluating Epoch 2:   0%|          | 0/77 [00:00<?, ?it/s]

Epoch 2/3, Train Loss: 0.2617, Validation Loss: 0.1375
Epoch 3/3


Training Epoch 3:   0%|          | 0/305 [00:00<?, ?it/s]

Evaluating Epoch 3:   0%|          | 0/77 [00:00<?, ?it/s]

Epoch 3/3, Train Loss: 0.1536, Validation Loss: 0.0807


In [ ]:
# Load the model for testing
model.load_state_dict(torch.load('transformer_regressor.pth'))
model.to(device)



TransformerRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
# Testing function with progress bar
def test_model(model, test_loader, device):
    model.eval()
    predictions = []
    actuals = []
    progress_bar = tqdm(test_loader, desc='Testing')
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device).unsqueeze(1)
            outputs = model(input_ids, attention_mask)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(targets.cpu().numpy())
    return predictions, actuals



In [ ]:
# Test the model
predictions, actuals = test_model(model, test_loader, device)

# Convert predictions and actuals to a DataFrame for better visualization
results_df = pd.DataFrame({
    'Predicted': [pred[0] for pred in predictions],
    'Actual': [act[0] for act in actuals]
})

print("First few rows of the results:")
display(results_df.head())

Testing:   0%|          | 0/77 [00:00<?, ?it/s]

First few rows of the results:


,Predicted,Actual
0,-0.133612,0.06
1,0.092350,0.12
2,3.914303,3.49
3,1.879154,1.77
4,1.939848,2.06


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Define the model class
class TransformerRegressor(torch.nn.Module):
    def __init__(self):
        super(TransformerRegressor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        return linear_output

# Load the trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerRegressor()
model.load_state_dict(torch.load('transformer_regressor.pth', map_location=device))
model.to(device)
model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to preprocess the input text
def preprocess(text, tokenizer, max_len=128):
    inputs = tokenizer(
        text=text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return inputs['input_ids'], inputs['attention_mask']



In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

# Assuming you have defined CustomDataset, TransformerRegressor, train_model, and evaluate_model

# Define a function to preprocess data
def preprocess_data(text, tokenizer, max_len=128):
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return inputs['input_ids'], inputs['attention_mask']

# Define a function to generate future dates
def generate_future_dates(start_date, periods):
    return pd.date_range(start=start_date, periods=periods, freq='D')

# Define a function to make predictions
def predict_future(model, tokenizer, start_date, periods, device):
    model.eval()
    future_dates = generate_future_dates(start_date, periods)
    future_data = [str(date) for date in future_dates]
    predictions = []

    for text in tqdm(future_data, desc="Predicting"):
        input_ids, attention_mask = preprocess_data(text, tokenizer)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask)
        predictions.append(outputs.cpu().numpy().flatten()[0])

    return future_dates, predictions

# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TransformerRegressor()
model.load_state_dict(torch.load('transformer_regressor.pth'))
model.to(device)

start_date = '2024-05-01'  # Example start date
periods = 365  # Number of days to predict

future_dates, future_predictions = predict_future(model, tokenizer, start_date, periods, device)

# Creating a DataFrame for better visualization
prediction_df = pd.DataFrame({
    'Date': future_dates,
    'Prediction': future_predictions
})

print(prediction_df)


Predicting:   0%|          | 0/365 [00:00<?, ?it/s]

          Date  Prediction
0   2024-05-01    1.404044
1   2024-05-02    1.351003
2   2024-05-03    1.445482
3   2024-05-04    1.546413
4   2024-05-05    1.559370
..         ...         ...
360 2025-04-26    1.512959
361 2025-04-27    1.508344
362 2025-04-28    1.406057
363 2025-04-29    1.405570
364 2025-04-30    1.415444

[365 rows x 2 columns]


In [ ]:
# Save the predictions to a CSV file
csv_file_path = 'predictions_May2024_1year.csv'
prediction_df.to_csv(csv_file_path, index=False)

print(f"Predictions saved to {csv_file_path}")

Predictions saved to predictions_May2024_1year.csv
